# Folded Spring

Folded-spring element: anchor + flexure arm + triangular terminal. Provides restoring force for the MEMS shuttle.

In [ ]:
import sys
sys.path.insert(0, "../../libraries")

import gdsfactory as gf
from mcw_custom_optical_mems_pdk import LAYER, PDK

PDK.activate()

In [ ]:
def make_spring(
    anchor_w: float = 5.0,
    anchor_h: float = 5.0,
    chevron_ext: float = 2.5,
    psg_inset: float = 1.0,
    arm_width: float = 0.5,
    arm_length: float = 62.0,
    arm_gap: float = 1.5,
    terminal_depth: float = 2.5,
    terminal_overlap: float = 1.0,
    struct_layer=LAYER.POLY_MEMS,
    sac_layer=LAYER.OXIDE_PSG,
    anchor_poly_layer=LAYER.POLY_ANCHOR,
    anchor_si_layer=LAYER.SI_FULL,
):
    """Folded-spring element: anchor + flexure arm + triangular terminal.

    Geometry (left to right):
        Anchor: rectangle (anchor_w x anchor_h) with a chevron tip extending
                chevron_ext past the right edge.  PSG/UDPOLY/SOI anchor pads
                are placed underneath.
        Arm:    Thin beam of width arm_width starting arm_gap past the anchor
                right edge, running for arm_length.
        Terminal: Right-pointing triangle connecting the arm tip to the shuttle.

    Args:
        anchor_w:          Anchor base rectangle width (um).
        anchor_h:          Anchor base rectangle height (um).
        chevron_ext:       How far the chevron tip extends past anchor_w (um).
        psg_inset:         Inset of PSG release window from anchor edges (um).
        arm_width:         Width of the flexure arm (um).
        arm_length:        Length of the flexure arm (um).
        arm_gap:           Gap between anchor right edge and arm start (um).
        terminal_depth:    X-extent of the triangular terminal (um).
        terminal_overlap:  How far the terminal tip overlaps back over the arm end (um).
        struct_layer:      Layer for structural poly-Si.
        sac_layer:         Layer for sacrificial PSG.
        anchor_poly_layer: Layer for undoped anchor poly.
        anchor_si_layer:   Layer for SOI anchor.
    """
    spring = gf.Component()
    mid_y = anchor_h / 2

    # --- Anchor ---
    anchor = gf.Component()
    # Pentagon: rectangle + chevron tip
    anchor.add_polygon(
        [(0, 0),
         (anchor_w, 0),
         (anchor_w + chevron_ext, mid_y),
         (anchor_w, anchor_h),
         (0, anchor_h)],
        layer=struct_layer,
    )
    # PSG release window
    anchor.add_polygon(
        [(psg_inset, psg_inset),
         (anchor_w - psg_inset, psg_inset),
         (anchor_w - psg_inset, anchor_h - psg_inset),
         (psg_inset, anchor_h - psg_inset)],
        layer=sac_layer,
    )
    # Anchor pads (full rectangle on undoped poly and SOI)
    anchor_rect = [(0, 0), (anchor_w, 0), (anchor_w, anchor_h), (0, anchor_h)]
    anchor.add_polygon(anchor_rect, layer=anchor_poly_layer)
    anchor.add_polygon(anchor_rect, layer=anchor_si_layer)

    # --- Flexure arm ---
    arm = gf.Component()
    arm_x0 = anchor_w + arm_gap
    arm_y0 = mid_y - arm_width / 2
    arm_x1 = arm_x0 + arm_length
    arm_y1 = arm_y0 + arm_width
    arm.add_polygon(
        [(arm_x0, arm_y0), (arm_x1, arm_y0), (arm_x1, arm_y1), (arm_x0, arm_y1)],
        layer=struct_layer,
    )

    # --- Triangular terminal ---
    terminal = gf.Component()
    tip_x = arm_x1 - terminal_overlap
    end_x = tip_x + terminal_depth
    terminal.add_polygon(
        [(tip_x, mid_y), (end_x, 0), (end_x, anchor_h)],
        layer=struct_layer,
    )

    spring.add_ref(anchor)
    spring.add_ref(arm)
    spring.add_ref(terminal)
    return spring

In [ ]:
spring = make_spring()
spring.plot()